In [82]:
%run utils.py
from functools import reduce

In [83]:
test = """\
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

#regex_input = "(\w+) (\w+) bags contain (\d+) (\w+) (\w+) ([bag?s?,?.]+) (\d+) (\w+) (\w+) ([bag?s?,?.]+)"
#regex_input = "(\w+) (\w+) bags contain (\d+) (\w+) (\w+) ([bag?s?,?.]+)"
#regex_input = "(\w+) (\w+) bags contain no other ([bag?s?,?.]+)"
regex_input = "(\w+) (\w+) bags contain|(\d+) (\w+) (\w+) ([bag?s?,?.]+)|(no other bags.)"

def test_reg(text:str):
    for s in text.splitlines():
        yield re.findall(regex_input,s)

list(test_reg(test))


[[('light', 'red', '', '', '', '', ''),
  ('', '', '1', 'bright', 'white', 'bag,', ''),
  ('', '', '2', 'muted', 'yellow', 'bags.', '')],
 [('dark', 'orange', '', '', '', '', ''),
  ('', '', '3', 'bright', 'white', 'bags,', ''),
  ('', '', '4', 'muted', 'yellow', 'bags.', '')],
 [('bright', 'white', '', '', '', '', ''),
  ('', '', '1', 'shiny', 'gold', 'bag.', '')],
 [('muted', 'yellow', '', '', '', '', ''),
  ('', '', '2', 'shiny', 'gold', 'bags,', ''),
  ('', '', '9', 'faded', 'blue', 'bags.', '')],
 [('shiny', 'gold', '', '', '', '', ''),
  ('', '', '1', 'dark', 'olive', 'bag,', ''),
  ('', '', '2', 'vibrant', 'plum', 'bags.', '')],
 [('dark', 'olive', '', '', '', '', ''),
  ('', '', '3', 'faded', 'blue', 'bags,', ''),
  ('', '', '4', 'dotted', 'black', 'bags.', '')],
 [('vibrant', 'plum', '', '', '', '', ''),
  ('', '', '5', 'faded', 'blue', 'bags,', ''),
  ('', '', '6', 'dotted', 'black', 'bags.', '')],
 [('faded', 'blue', '', '', '', '', ''),
  ('', '', '', '', '', '', 'no other 

In [90]:
regex_input = """\
(?P<agg>\w+) (?P<col>\w+) bags contain|\
(?P<num>\d+) (?P<cagg>\w+) (?P<ccol>\w+) ([bag?s?,?.]+)|\
(?P<no_bags>no other bags.)"""

def parse(iter_in):
    for s in iter_in :
        parsed_list = [mo.groupdict() for mo in re.finditer(regex_input, s)]
        k = "{agg}_{col}".format(**parsed_list[0])
        if len(parsed_list) == 2 and parsed_list[1]['no_bags']:
            yield { k: None }
        else :
            yield { k: {"{cagg}_{ccol}".format(**d): d['num'] for d in parsed_list[1:]} }

d = dict()
for dd in parse(test.splitlines()):
    d.update(dd)
d

{'light_red': {'bright_white': '1', 'muted_yellow': '2'},
 'dark_orange': {'bright_white': '3', 'muted_yellow': '4'},
 'bright_white': {'shiny_gold': '1'},
 'muted_yellow': {'shiny_gold': '2', 'faded_blue': '9'},
 'shiny_gold': {'dark_olive': '1', 'vibrant_plum': '2'},
 'dark_olive': {'faded_blue': '3', 'dotted_black': '4'},
 'vibrant_plum': {'faded_blue': '5', 'dotted_black': '6'},
 'faded_blue': None,
 'dotted_black': None}

In [123]:
def part_1(iter_in):
    bags = dict()
    for d in parse(iter_in):
        bags.update(d)

    shiny_gold = set()
    search_stack = ["shiny_gold"]
    while len(search_stack):
        what = search_stack.pop()
        for k,v in bags.items():
            if v and what in v.keys():
                shiny_gold.add(k)
                search_stack.append(k)
    
    return shiny_gold

len(part_1(open("input/input7.txt")))
#part_1(test.splitlines())

332

In [122]:
test = """\
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags."""

d = dict()
for dd in parse(test.splitlines()):
    d.update(dd)
d

{'shiny_gold': {'dark_red': '2'},
 'dark_red': {'dark_orange': '2'},
 'dark_orange': {'dark_yellow': '2'},
 'dark_yellow': {'dark_green': '2'},
 'dark_green': {'dark_blue': '2'},
 'dark_blue': {'dark_violet': '2'},
 'dark_violet': None}

In [142]:
def part_2(iter_in):
    bags = dict()
    for d in parse(iter_in):
        bags.update(d)

    shiny_gold = []
    search_stack = [("shiny_gold",1)]
    while len(search_stack):
        what,cnt = search_stack.pop()
        if bags[what] :
            aux = []
            for k,v in bags[what].items():
                aux.append(int(v))
                search_stack.append((k,v))
            shiny_gold.append(int(cnt)+(int(cnt)*sum(aux)))

    return shiny_gold


In [143]:
part_2(test.splitlines())

[3, 6, 6, 6, 6, 6]

In [139]:
part_2(open("input/input7.txt"))

[['3', '2', '4'],
 ['1'],
 ['1', '5', '1', '1'],
 ['1', '4', '3', '3'],
 ['3', '5', '4', '5'],
 ['5', '5', '4'],
 ['1'],
 ['5', '1', '5', '1'],
 ['3', '3'],
 ['3', '5', '4', '5'],
 ['1', '1', '4', '5'],
 ['2', '4', '4', '2'],
 ['3', '3'],
 ['4', '2'],
 ['2', '4', '4', '2'],
 ['5', '3', '5'],
 ['2', '4'],
 ['3', '3'],
 ['2'],
 ['3', '2'],
 ['2'],
 ['3', '3'],
 ['4', '2'],
 ['2', '4', '4', '2'],
 ['5', '3', '5'],
 ['5', '5', '4'],
 ['1'],
 ['5', '1', '5', '1'],
 ['3', '3'],
 ['3', '5', '4', '5'],
 ['1', '1', '4', '5'],
 ['2', '4', '4', '2'],
 ['3', '3'],
 ['4', '2'],
 ['2', '4', '4', '2'],
 ['5', '3', '5']]

In [128]:
bags["shiny_gold"]

{'clear_fuchsia': '3', 'vibrant_indigo': '2', 'dotted_maroon': '4'}